# **MAC0460 - EP2**
### **Nome:** Gustavo Nogai Saito.
### **NUSP:** 10730021.
&nbsp;

In [94]:
from google.colab import files
import pandas as pd
import io
import numpy as np

#### **Upload local do dataset "pokemon.csv" pelo google colab:**

In [ ]:
uploaded = files.upload()

df = pd.read_csv(io.BytesIO(uploaded['pokemon.csv']))

#### **Trata o csv, escolhendo os atributos relevantes, removendo valores *NaN* e criando o vetor das labels *y*:**

In [96]:
x = df.drop(['abilities', 'against_bug', 'against_dark', 'against_dragon', 'against_fairy', 'against_fight', 'against_fire', 'against_flying', 'against_ghost', 'against_grass', 'against_ground', 'against_ice', 'against_normal', 'against_poison', 'against_psychic', 'against_rock', 'against_steel', 'against_water', 'base_egg_steps', 'base_happiness', 'capture_rate', 'classfication', 'type2', 'experience_growth', 'japanese_name', 'name', 'percentage_male', 'pokedex_number', 'generation', 'is_legendary'], inplace=False, axis=1)

for col in x:
  if col == 'type1':
    continue
  for i, row in x[col].items():
    if np.isnan(x[col][i]):
      x = x.drop(index=i)

y = []
for i, row in (x.iterrows()):
  if row.type1 == 'water' or row.type1 == 'normal':
    y.append(1)
  else:
    y.append(0)

x = x.drop(['type1'], inplace=False, axis=1)


#### **Importa funções comuns aos modelos testados e define a seed para aleatorização:**

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

random_seed = None

#### **Separa 90% dos dados para fazer a escolha dos melhores parâmetros e 10% para escolher o melhor modelo**:

In [98]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=random_seed)

# **Para testarmos os parâmetros usaremos o *GridSearchCV* do *sklearn* como recomendado no enunciado.**

## **Regressão Logística:**

In [99]:
from sklearn.linear_model import LogisticRegression

parameters = {'solver':('lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga')}

logReg = LogisticRegression(random_state=random_seed, max_iter=10000)

grid = GridSearchCV(logReg, parameters, cv=10, n_jobs=-1, return_train_score=True)

grid.fit(x_train, y_train)

logReg_best_solver = grid.best_params_['solver']

print('Melhor parâmetro de regularização: ' + logReg_best_solver)
print(' Acurácia in:  ' + str(grid.cv_results_['mean_train_score'][grid.best_index_]))
print(' Acurácia out: ' + str(grid.cv_results_['mean_test_score'][grid.best_index_]))

Melhor parâmetro de regularização: liblinear
 Acurácia in:  0.8075362595037012
 Acurácia out: 0.802112676056338


## **Support Vector Machine:**

In [100]:
from sklearn.svm import SVC

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid')}

svc = SVC(max_iter=-1, random_state=random_seed)

grid = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, return_train_score=True)

grid.fit(x_train, y_train)

svc_best_kernel = grid.best_params_['kernel']

print('Melhor kernel: ' + svc_best_kernel)
print(' Acurácia in:  ' + str(grid.cv_results_['mean_train_score'][grid.best_index_]))
print(' Acurácia out: ' + str(grid.cv_results_['mean_test_score'][grid.best_index_]))

Melhor kernel: linear
 Acurácia in:  0.8097504463479709
 Acurácia out: 0.8105633802816902


## **Decision Tree:**

In [101]:
from sklearn.tree import DecisionTreeClassifier

parameters = {'max_depth':(1, 2, 3, 4, 5, 6, 7), 'min_samples_leaf':(1, 2, 3, 4, 5, 6, 7)}

dtc = DecisionTreeClassifier(random_state=random_seed)

grid = GridSearchCV(dtc, parameters, cv=10, n_jobs=-1, return_train_score=True)

grid.fit(x_train, y_train)

dtc_best_depth = grid.best_params_['max_depth']
dtc_best_minLeaf = grid.best_params_['min_samples_leaf']

print('Melhor profundidade da árvore: ' + str(dtc_best_depth))
print('Melhor mínimo de amostras para ser uma folha: ' + str(dtc_best_minLeaf))
print(' Acurácia in:  ' + str(grid.cv_results_['mean_train_score'][grid.best_index_]))
print(' Acurácia out: ' + str(grid.cv_results_['mean_test_score'][grid.best_index_]))

Melhor profundidade da árvore: 1
Melhor mínimo de amostras para ser uma folha: 1
 Acurácia in:  0.8091170334409918
 Acurácia out: 0.8091348088531187


## **Random Forest:**

In [102]:
from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators':(50, 100, 150, 200), 'max_depth':(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), 'min_samples_leaf':(1, 2, 3, 4, 5, 6, 7)}

rfc = RandomForestClassifier(n_jobs=-1, random_state=random_seed)

grid = GridSearchCV(rfc, parameters, cv=10, n_jobs=-1, return_train_score=True)

grid.fit(x_train, y_train)

rfc_best_nEstimators = grid.best_params_['n_estimators']
rfc_best_depth = grid.best_params_['max_depth']
rfc_best_minLeaf = grid.best_params_['min_samples_leaf']

print('Melhor número de estimadores: ' + str(rfc_best_nEstimators))
print('Melhor profundidade da árvore: ' + str(rfc_best_depth))
print('Melhor mínimo de amostras para ser uma folha: ' + str(rfc_best_minLeaf))
print(' Acurácia in:  ' + str(grid.cv_results_['mean_train_score'][grid.best_index_]))
print(' Acurácia out: ' + str(grid.cv_results_['mean_test_score'][grid.best_index_]))

Melhor número de estimadores: 200
Melhor profundidade da árvore: 7
Melhor mínimo de amostras para ser uma folha: 6
 Acurácia in:  0.8573915725490983
 Acurácia out: 0.806277665995976


## **Extra 1 - Regressão Linear:**

In [103]:
from sklearn.linear_model import LinearRegression

parameters = {'fit_intercept':(True, False)}

linReg = LinearRegression()

grid = GridSearchCV(linReg, parameters, cv=10, n_jobs=-1, return_train_score=True)

grid.fit(x_train, y_train)

linReg_best_intercept = grid.best_params_['fit_intercept']

print('Melhor opção para o intercept: ' + str(grid.best_params_['fit_intercept']))
print(' Acurácia in:  ' + str(grid.cv_results_['mean_train_score'][grid.best_index_]))
print(' Acurácia out: ' + str(grid.cv_results_['mean_test_score'][grid.best_index_]))

Melhor opção para o intercept: False
 Acurácia in:  0.29650135512743325
 Acurácia out: 0.27897545846151983


## **Extra 2 - Perceptron:**

In [104]:
from sklearn.linear_model import Perceptron

parameters = {'penalty':(None,'l2','l1','elasticnet')}

percep = Perceptron(random_state=random_seed)

grid = GridSearchCV(percep, parameters, cv=10, n_jobs=-1, return_train_score=True)

grid.fit(x_train, y_train)

percep_best_penalty = grid.best_params_['penalty']

print('Melhor parâmetro de regularização: ' + grid.best_params_['penalty'])
print(' Acurácia in:  ' + str(grid.cv_results_['mean_train_score'][grid.best_index_]))
print(' Acurácia out: ' + str(grid.cv_results_['mean_test_score'][grid.best_index_]))

Melhor parâmetro de regularização: elasticnet
 Acurácia in:  0.6783162651206645
 Acurácia out: 0.6729979879275654


## **Escolhendo o melhor modelo com os melhores parâmetros já escolhidos:**

### Aqui usaremos a parte do dataset reservada (10%) como teste final para ver qual modelo possui maior acurácia:

In [105]:
modelos = ['Logistic_Regression', 'Support_Vector_Machine', 'Decision_Tree', 'Random_Forest', 'Linear_Regression', 'Perceptron']

acuracias = []
modelos_testados = []

for modelo in modelos:
  if modelo == 'Logistic_Regression':
    clf = LogisticRegression(solver=logReg_best_solver, random_state=random_seed, max_iter=10000)

  elif modelo == 'Support_Vector_Machine':
    clf = SVC(kernel=svc_best_kernel, max_iter=-1, random_state=random_seed)

  elif modelo == 'Decision_Tree':
    clf = DecisionTreeClassifier(max_depth=dtc_best_depth, min_samples_leaf=dtc_best_minLeaf, random_state=random_seed)

  elif modelo == 'Random_Forest':
    clf = RandomForestClassifier(n_estimators=rfc_best_nEstimators, max_depth=rfc_best_depth, min_samples_leaf=rfc_best_minLeaf, n_jobs=-1, random_state=random_seed)

  elif modelo == 'Linear_Regression':
    clf = LinearRegression(fit_intercept=linReg_best_intercept)

  elif modelo == 'Perceptron':
    clf = Perceptron(penalty=percep_best_penalty, random_state=random_seed)

  clf.fit(x_test, y_test)

  acuracias.append(clf.score(x_test, y_test))
  modelos_testados.append(modelo)

### Vamos ordenar em order decrecente as acurácias de cada modelo testado:

In [106]:
idx = np.argsort(acuracias)

modelos_cresc = np.array(modelos_testados)[idx]
modelos_dec = modelos_cresc[::-1]

acuracias_cresc = np.array(acuracias)[idx]
acuracias_dec = acuracias_cresc[::-1]

for i in range(len(acuracias)):
  print('\033[1m' + "{:>23}".format(modelos_dec[i] + ':') + '\033[0m' + '  Acurácia = ' + str(acuracias_dec[i]))
  print()

         Random_Forest:  Acurácia = 0.8734177215189873

Support_Vector_Machine:  Acurácia = 0.8227848101265823

   Logistic_Regression:  Acurácia = 0.7721518987341772

         Decision_Tree:  Acurácia = 0.759493670886076

            Perceptron:  Acurácia = 0.43037974683544306

     Linear_Regression:  Acurácia = 0.2994870414381474



### Logo, podemos ver que o modelo com melhor acurácia nos testes realizados foi:

In [107]:
print('O modelo com maior acurácia foi o \033[1m' + modelos_dec[0] + '\033[0m, apresentando uma acurácia de ' + str(acuracias_dec[0]) + '.')

O modelo com maior acurácia foi o Random_Forest, apresentando uma acurácia de 0.8734177215189873.
